In [35]:
!pip install bertopic
!pip install -U FlagEmbedding
!pip install -U sentence-transformers
!pip install classla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.8/407.8 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Su

In [36]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import nltk
import classla
import re

from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from FlagEmbedding import BGEM3FlagModel
from bertopic import BERTopic
from umap import UMAP

The csv in the next cell has already been processed using nltk. First I tried to run a classical multilingual BERTopic and a BGE embedding BERTopic on a small subset.

In [10]:
df = pd.read_csv("/content/Corpus.csv")

In [11]:
topic_model = BERTopic(language="multilingual")

In [12]:
topics, probs = topic_model.fit_transform(df["clean"])

In [13]:
reps = topic_model.get_topic_info()["Representation"]

In [ ]:
topic_model.visualize_barchart(tmop_n_topics = 16, n_words = 10)

In [ ]:
topic_model.visualize_topics()

In [18]:
from wordcloud import WordCloud

def create_wordcloud(model, topic):
    text = {word: value for word, value in topic_model.get_topic(topic)}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()

# Show wordcloud
create_wordcloud(topic_model, topic=2) #choose the number of the topic to see the words

TypeError: 'bool' object is not iterable

***Trying out the model with BGE-M3 does not seem to work on this Collab environment***

In [15]:
from sentence_transformers import SemntenceTransformer

emb = SentenceTransformer("BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [16]:
#mod = emb.encode(df["clean"][:10])

In [25]:
topic_model_2 = BERTopic(embedding_model=emb)
top, pro = topic_model_2.fit_transform(df["clean"][:10])

In [28]:
reps_2 = topic_model_2.get_topic_info()["Representation"]

Now, try to do the preprocessing using classla pipeline for slovenian and repeat the process. You should run BERTopic on df["classla"] this time.

classla.download("sl")
nlp = classla.Pipeline('sl', processors='tokenize, pos, lemma')

In [45]:
classla = []
for i in range(10):
  classla.append(nlp(df["clean"][i]))